In [19]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import speech_recognition as sr
import pyttsx3
import openai
import requests
import json
from bs4 import BeautifulSoup as bs
from difflib import SequenceMatcher
import subprocess


In [20]:

openai.api_key_path = "key.txt"
cur_model = "text-davinci-002"
img = ["image", "picture", "photo", "landscape", "painting", "drawing", "art"]


# Define Brave path
brave_path = "C:/Program Files/BraveSoftware/Brave-Browser-Nightly/Application/brave.exe"
options = webdriver.ChromeOptions()
options.binary_location = brave_path

# set dl options
prefs = {"download.default_directory": "C:/Users/thewa/Desktop/projects/computational_neuroscience/AI_ML/projects/mun_ching/crawlers/content"}

options.add_experimental_option("prefs", prefs)

# Create new automated instance of Brave
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 26.2MB/s]
C:\Users\thewa\AppData\Local\Temp\ipykernel_20976\1923631790.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [21]:
def callGPT(prompt):

    completion = openai.Completion.create(
        model=cur_model,
        prompt=prompt,
        temperature=0,
        frequency_penalty=0.5,
        max_tokens=20,


    )

    return completion.choices[0].text


In [22]:

def getGame(response):

    URL = "https://steamcommunity.com/id/yusufwadi/games?tab=all&xml=1"
    data = requests.get(url=URL)
    soup = bs(data.text, 'xml')
    names = soup.find_all("name")
    response = response.lower()

    for name in names:

        if ((SequenceMatcher(a=response, b=name.text.lower()).ratio()) > 0.70):
            engine.say("Now launching... " + name.text)
            engine.runAndWait()
            print(name.text)
            return name.parent.find("appID").text

    print("Game not found")

    # print("hello")
#getGame("no mans sky")


In [23]:
def speak(tts):

    with sr.Microphone() as source:
        engine.say(tts)
        engine.runAndWait()
        print('')
        audio = r3.listen(source)
        keyword = r3.recognize_google(audio)

        YOUR_PROMPT = "Extract a search query from the following prompt: " + '"' + keyword + '"'

        response = callGPT(YOUR_PROMPT)

        return response


In [24]:
def listening():
    with sr.Microphone() as source:
        r3.adjust_for_ambient_noise(source)
        engine.say("What can I do for you today?")
        engine.runAndWait()
        audio = r3.listen(source, phrase_time_limit=3)
        phrase = r3.recognize_google(audio)
        return phrase

In [25]:

# From here

def qCommand(x):

    x = x.lower()
    r3 = sr.Recognizer()

    if 'youtube' in x:

        driver.get('https://www.youtube.com/')

        response = speak("What do you want to see?")

        YOUR_PROMPT = "Extract a search query from the following prompt: " + '"' + keyword + '"'

        response = callGPT(YOUR_PROMPT)

        elem = driver.find_element('xpath', '//input[@id="search"]')
        elem.click()
        elem.send_keys(response, Keys.RETURN)

        # tryAudio(audio)

        # Till here is the code to run a YouTube vid
    elif 'google' in x:

        driver.get('https://www.google.com/')

        with sr.Microphone() as source:
            engine.say("What do you want to search for?")
            engine.runAndWait()
            print('')
            audio = r3.listen(source, timeout=5)
            keyword = r3.recognize_google(audio)
            print(keyword)
        YOUR_PROMPT = "Extract a search query from the following prompt: " + '"' + keyword + '"'

        response = callGPT(YOUR_PROMPT)

        elem = driver.find_element('xpath', '//input[@class="gLFyf gsfi"]')
        elem.click()
        elem.send_keys(response, Keys.RETURN)

        # tryAudio(audio)

        # if img in x:

    elif 'play' in x:

        print(x)

        prompt = "Extract the name of the game from the prompt: " + x

        response = callGPT(prompt)

        gameID = getGame(response)

        subprocess.call([r'steam.bat', gameID])

        # tryAudio(audio)

    else:
        print("command not recognized")


In [26]:

r3 = sr.Recognizer()

engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)
noCom = True

print('Listening...')
engine.say("Hello. I am Idaw. Hold for calibration...")
engine.runAndWait()

phrase = listening()
print(phrase)
qCommand(phrase)


Listening...
I want to play Risk of Rain 2
i want to play risk of rain 2
Risk of Rain 2


# Voice Assist